In [ ]:
#import library

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
#data

data = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3],
    'item_id': [101, 102, 101, 103, 102, 104],
    'rating': [5, 4, 5, 3, 4, 2]
})

data.head(5)

,user_id,item_id,rating
0,1,101,5
1,1,102,4
2,2,101,5
3,2,103,3
4,3,102,4


In [ ]:
raw_data = pd.read_csv('eco_rating-1.csv')
data = raw_data[:800]
data

,user_id,item_id,ratings
0,1,28,2
1,1,86,4
2,1,94,4
3,1,146,3
4,2,17,2
...,...,...,...
795,145,170,3
796,145,13,3
797,145,35,2
798,146,40,5


In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# ini data training (80%) sebanyak 640 data

train

,user_id,item_id,ratings
264,49,147,3
615,111,118,3
329,61,105,3
342,63,93,4
394,72,155,5
...,...,...,...
71,13,54,3
106,21,90,2
270,50,162,2
435,79,75,5


In [ ]:
# ini data test (20%) sebanyak 160 data

test

,user_id,item_id,ratings
696,127,3,3
667,121,76,2
63,12,47,4
533,97,27,5
66,12,156,3
...,...,...,...
589,106,62,4
798,146,40,5
744,136,129,5
513,93,64,3


In [ ]:
#menghitung total jumlah user dan item yang unique

number_users = data['user_id'].nunique()
number_items = data['item_id'].nunique()

number_users, number_items


(146, 181)

In [ ]:
#create model NCF

user_input = tf.keras.layers.Input(shape=(1,))
item_input = tf.keras.layers.Input(shape=(1,))

user_embedding = tf.keras.layers.Embedding(number_users + 10, 16)(user_input)
item_embedding = tf.keras.layers.Embedding(number_items + 10, 16)(item_input)

user_flat = tf.keras.layers.Flatten()(user_embedding)
item_flat = tf.keras.layers.Flatten()(item_embedding)

concat = tf.keras.layers.Concatenate()([user_flat, item_flat])
dropout_1 = tf.keras.layers.Dropout(0.5)(concat)
dense_1 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(dropout_1)
dropout_2 = tf.keras.layers.Dropout(0.3)(dense_1)
dense_2 = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(dropout_2)
output = tf.keras.layers.Dense(1)(dense_2)

model = tf.keras.models.Model(inputs=[user_input, item_input], outputs=output)

model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_23 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 input_24 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 embedding_22 (Embedding)    (None, 1, 16)                2496      ['input_23[0][0]']            
                                                                                                  
 embedding_23 (Embedding)    (None, 1, 16)                3056      ['input_24[0][0]']            
                                                                                           

In [ ]:
#model compiler with optimizer and loss mse

model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [ ]:
#training data with model

model.fit([train['user_id'], train['item_id']], train['ratings'], epochs=100, batch_size=32, validation_split=0.2, verbose=2)

Epoch 1/100
16/16 - 1s - loss: 12.1825 - val_loss: 8.7538 - 1s/epoch - 83ms/step
Epoch 2/100
16/16 - 0s - loss: 6.8709 - val_loss: 3.8744 - 102ms/epoch - 6ms/step
Epoch 3/100
16/16 - 0s - loss: 2.9758 - val_loss: 2.1277 - 89ms/epoch - 6ms/step
Epoch 4/100
16/16 - 0s - loss: 2.0897 - val_loss: 1.9785 - 96ms/epoch - 6ms/step
Epoch 5/100
16/16 - 0s - loss: 1.8602 - val_loss: 1.8873 - 100ms/epoch - 6ms/step
Epoch 6/100
16/16 - 0s - loss: 1.6948 - val_loss: 1.7875 - 104ms/epoch - 7ms/step
Epoch 7/100
16/16 - 0s - loss: 1.5755 - val_loss: 1.7466 - 71ms/epoch - 4ms/step
Epoch 8/100
16/16 - 0s - loss: 1.4569 - val_loss: 1.7976 - 76ms/epoch - 5ms/step
Epoch 9/100
16/16 - 0s - loss: 1.3953 - val_loss: 1.6626 - 63ms/epoch - 4ms/step
Epoch 10/100
16/16 - 0s - loss: 1.2854 - val_loss: 1.6521 - 62ms/epoch - 4ms/step
Epoch 11/100
16/16 - 0s - loss: 1.2450 - val_loss: 1.6398 - 87ms/epoch - 5ms/step
Epoch 12/100
16/16 - 0s - loss: 1.1430 - val_loss: 1.6200 - 62ms/epoch - 4ms/step
Epoch 13/100
16/16 - 0

In [ ]:
#evaluate model with test data

result = model.evaluate([test['user_id'], test['item_id']], test['rating'])
result